# Applied Machine Learning Challenge 
University of Trento, A.y. 2020-2021 \
Roosters Team


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [10]:
#Define roots to the train and test folders used throughout the notebook
#google landmarks data
training_root = "/content/drive/.shortcut-targets-by-id/15RXzXpljEuRWdjQPcx4Kd8EDKWgjujOt/AML/GoogleLandmarksSubset/training"
#competition data root
test_root = "/content/drive/.shortcut-targets-by-id/0B5NgX9ua1kQkfmxseGVTVDVuSDROaU1EMFpZUTRvWU9pREx6eXJTSVBHLWFKYmVhT2R6Tjg/Applied Machine Learning LM Data Science/Challenge/dataset/test"
#Arbitrary path to save model
model_save_dir = '/content/drive/.shortcut-targets-by-id/15RXzXpljEuRWdjQPcx4Kd8EDKWgjujOt/AML/AMLretrievalv1.pth'
#Data provided by instructors at the beginning of the couse to validate our models 
validation_query_root = '/content/drive/.shortcut-targets-by-id/0B5NgX9ua1kQkfmxseGVTVDVuSDROaU1EMFpZUTRvWU9pREx6eXJTSVBHLWFKYmVhT2R6Tjg/Applied Machine Learning LM Data Science/Challenge/dataset/validation/query'
validation_gallery_root = '/content/drive/.shortcut-targets-by-id/0B5NgX9ua1kQkfmxseGVTVDVuSDROaU1EMFpZUTRvWU9pREx6eXJTSVBHLWFKYmVhT2R6Tjg/Applied Machine Learning LM Data Science/Challenge/dataset/validation/gallery'

###Dataset and Dataloader

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import os 
from skimage import io
from PIL import Image


class AMLtestImages(Dataset):
  """
  This class is meant to handle the images released during the competition day for AML 2021.

  Arguments:
  parentdir: should contain two folders: gallery and query, containing each only image files.
  split: dataset to be instantiated, either gallery or query, as a string. 
  """
  def __init__(self,parentdir,split,transform=None):
    self.split = split
    self.transform = transform
    self.root_dir = parentdir
    self.gallery = sorted(list(os.listdir(os.path.join(parentdir, 'gallery'))))
    self.query = sorted(list(os.listdir(os.path.join(parentdir, 'query'))))
  
  def __getitem__(self,idx):
    if self.split == 'gallery':
      idximagepath = os.path.join(self.root_dir, 'gallery', self.gallery[idx]) #path to image #idx
      image = Image.open(idximagepath).convert('RGB')
      if self.transform:
            image = self.transform(image)
      return {'pic' : image, 'name' : self.gallery[idx]}

    if self.split == 'query':
      idximagepath = os.path.join(self.root_dir, 'query', self.query[idx]) #path to image #idx
      image = Image.open(idximagepath).convert('RGB')
      if self.transform:
            image = self.transform(image)
      return {'pic' : image, 'name' : self.query[idx]}

  def __len__(self):
    if self.split == 'gallery':
      return len(self.gallery )
    if self.split == 'query':
      return len(self.query )



class ImageFolderWithPaths(ImageFolder):
    """
    Custom dataset that returns images as well as their filenames. 
    Extends torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        imgname = path.split(os.sep)[-1]

        return {'pic' : original_tuple[0], 'name' : imgname}

In [4]:
import torch
from torchvision import transforms
import numpy as np


def get_data(training_root, test_root, batch_size, test_batch_size, validation_query_root=None,validation_gallery_root=None):

  # Transformations to be applied 
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  transform_train = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean, std),
                                       transforms.Resize((256,256)),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomCrop((224,224))])#cannot be lower than 224
  transform_test = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean, std),
                                       transforms.Resize((224,224))])#cannot be lower than 224
  
  # Dataset instances
  dataset_train = ImageFolder(training_root, transform = transform_train)
  dataset_gall = AMLtestImages(test_root,'gallery', transform = transform_test)
  dataset_query = AMLtestImages(test_root,'query', transform = transform_test)
  if validation_gallery_root and validation_query_root:
    dataset_val_query = ImageFolderWithPaths(validation_query_root, transform=transform_test)
    dataset_val_gall = ImageFolderWithPaths(validation_gallery_root, transform=transform_test)
  
  #size of last layer
  numclasses = len(dataset_train.classes)

  # Initialization of dataloaders
  train_loader = torch.utils.data.DataLoader(dataset_train, batch_size, shuffle=True)
  gall_loader = torch.utils.data.DataLoader(dataset_gall, test_batch_size, shuffle=False) 
  query_loader = torch.utils.data.DataLoader(dataset_query, test_batch_size, shuffle=False) 
  if validation_gallery_root and validation_query_root:
    query_validation_loader = torch.utils.data.DataLoader(dataset_val_query, test_batch_size, shuffle=False) 
    gall_validation_loader = torch.utils.data.DataLoader(dataset_val_gall, test_batch_size, shuffle=False) 
  else: 
    query_validation_loader = None
    gall_validation_loader = None


  
  return train_loader, gall_loader, query_loader, numclasses, query_validation_loader, gall_validation_loader

### Defining the network 

In [5]:
import torch.nn as nn
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
from torchvision.models.resnet import ResNet,Bottleneck,BasicBlock,model_urls
from torch.hub import load_state_dict_from_url
import torchvision
from torch.utils.tensorboard import SummaryWriter

def get_net(num_classes):
  myresnet = torchvision.models.resnet50(pretrained=True)
  # Substitute last layer with a FC of specified dimension  
  myresnet.fc = nn.Sequential(nn.Linear(2048,num_classes))

  return myresnet



###Train and utility functions

In [7]:
def get_optimizer(model, lr, momentum):
    
  # the newly initialized layer and the rest of the network need to be trained 
  # with a differentlearning rate 
  
  final_layer_weights = []
  rest_of_the_net_weights = []
  
  # layers of the network
  for name, param in model.named_parameters():
    if name.startswith('fc'):
      final_layer_weights.append(param)
    else:
      rest_of_the_net_weights.append(param)
  
  # training: 
  # final_layer_weights with learning_rate = lr
  # rest_of_the_net_weights with learning_rate = lr / 10
  
  optimizer = torch.optim.SGD([
      {'params': rest_of_the_net_weights},
      {'params': final_layer_weights, 'lr': lr}
  ], lr=lr / 10, momentum=momentum)
  
  return optimizer


def train(net,data_loader,optimizer,cost_function, device):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.
  
  net.train() 
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU if available
    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = net(inputs)

    # Apply the loss
    loss = cost_function(outputs,targets)
      
    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # metrics to be returned after each epoch
    samples+=inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


### Main

In [7]:
def main(batch_size=32,
         test_batch_size = 32, 
         learning_rate=0.001, 
         momentum=0.9, 
         epochs=50, 
         visualization_name='ResNet50', 
         training_root=None,
         test_root=None,
         save_dir=None):
  
  writer = SummaryWriter(log_dir="runs/exp1")
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  # Instantiates dataloaders
  train_loader, _, _, numclasses,_ , _ = get_data(training_root, test_root, batch_size = batch_size, test_batch_size = test_batch_size)
  
  # Instantiates the model
  net = get_net(numclasses).to(device)
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, momentum)
  
  # Instantiates the cost function
  cost_function = nn.CrossEntropyLoss()


  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function, device)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)


  # Closes the logger
  writer.close()
  # Save the model for later usage 
  torch.save(net.state_dict(), save_dir)

###Training the network and visualizing results 

In [ ]:
main(training_root=training_root, test_root=test_root, save_dir = model_save_dir)

In [ ]:
#visualize the tensorboard output 
%load_ext tensorboard
%tensorboard --logdir=runs

###Computing distances between gallery and query

In [16]:
from tqdm.notebook import tqdm
import requests
import json

#define a forward hook function to get embeddings from net
class SaveOutput():

  def __init__(self):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.outputs = torch.Tensor(0,2048).to(self.device)
      
  def __call__(self, module, module_in, module_out):
    self.outputs = torch.cat((self.outputs, module_out.squeeze()), dim=0)
      
  def clear(self):
    self.outputs = torch.Tensor((0,2048))


def get_mappings(net, gall_loader, query_loader):
  #Forward pass on all images
  with torch.no_grad():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    results = {}
    for phase in ['gallery','query']:
      img_embedding = SaveOutput()
      net.avgpool.register_forward_hook(img_embedding)
      img_names = []
      if phase == 'gallery':
        loader = gall_loader
      else:
        loader = query_loader
      
      for idx, sample in enumerate(tqdm(loader)):
        #Send obect to GPU if available
        sample['pic'] = sample['pic'].to(device)
        out = net(sample['pic'])
        img_names.extend(sample['name'])
      #Save the computed embeddings 
      print(f'--> Forward pass on {phase} images completed. Shape of output tensor: {img_embedding.outputs.shape}')
      results[phase] = {'emb': img_embedding.outputs, 'names': img_names}

  #Building distance matrix
  distances = torch.cdist(results['query']['emb'],results['gallery']['emb'])
  print(f'--> Distance matrix computed with size:{distances.size()}')
  indices = torch.argsort(distances, dim=1) #unit of analysis for sorting is the row

  #Finally build the dictionary with the results 
  mappings = {}
  for i in range(indices.shape[0]):#iterate over query length
    matches = []
    for j in range(indices.shape[1]):#iterate over gallery length
      #get correct matching gallery img name 
      current_gall_name = results['gallery']['names'][indices[i,j]]
      matches.append(current_gall_name)
    current_qry_name = results['query']['names'][i]
    mappings[current_qry_name] = matches

  return mappings



###Get results for validation


In [18]:
#If running as a Notebook, please execute previous cells 
def get_ground_truth(validation_query_root, validation_gallery_root):
  ret = {}
  for qry_folder in os.listdir(validation_query_root):
    try:
      matches = set(os.listdir(os.path.join(validation_gallery_root,qry_folder)))
    except:
      matches = set()
    for qry_img in os.listdir(os.path.join(validation_query_root,qry_folder)):
      ret[qry_img] = matches
  
  return ret
    
def topk(result,ground,k):
  correct = 0
  total = 0
  for qry in result: 
    total+=1
    for i in range(k): 
      #check if any of the top k matches is in ground truth
      if result[qry][i] in ground[qry]:
        correct+=1
        break
  return correct/total*100


_, _, _, numclasses, query_loader, gall_loader = get_data(training_root, test_root, 32, 32, validation_query_root, validation_gallery_root)
# Instantiating the network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = get_net(1394).to(device) #argument of get_net should be equal to numclasses (number of classes used for training the net)
net.load_state_dict(torch.load(model_save_dir,map_location=device))
computed_dic = get_mappings(net, gall_loader, query_loader)
truth_dic = get_ground_truth(validation_query_root, validation_gallery_root)

for k in [1,3,10]:
  print(f'Top {k} accuracy: {topk(computed_dic, truth_dic,k):.2f}')



--> Forward pass on gallery images completed. Shape of output tensor: torch.Size([534, 2048])



--> Forward pass on query images completed. Shape of output tensor: torch.Size([70, 2048])
--> Distance matrix computed with size:torch.Size([70, 534])
Top 1 accuracy: 90.00
Top 3 accuracy: 94.29
Top 10 accuracy: 94.29


###Get results for the competition day

In [19]:

#The submission fnction to get accuracy results 
def submit(results, url):
  res = json.dumps(results)
  response = requests.post(url, res)
  result = json.loads(response.text)
  print("accuracy is {}".format(result['results']))

#Instantiating dataloaders
_, gall_loader, query_loader, numclasses, _, _ =  get_data(training_root, test_root, 32, 32, validation_query_root, validation_gallery_root)

# Instantiating the network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = get_net(1394).to(device) #argument of get_net should be equal to numclasses (number of classes used for training the net)
#net.load_state_dict(torch.load(model_save_dir,map_location=device))

#Create the final submission product and obtain results 
group = dict()
group['groupname'] = "Roosters"
group['images'] = get_mappings(net, gall_loader, query_loader)
submit(group, 'http://ec2-18-191-24-254.us-east-2.compute.amazonaws.com/competition/')




--> Forward pass on gallery images completed. Shape of output tensor: torch.Size([408, 2048])



--> Forward pass on query images completed. Shape of output tensor: torch.Size([15, 2048])
--> Distance matrix computed with size:torch.Size([15, 408])
accuracy is {'top1': 0.4, 'top3': 0.4666666666666667, 'top10': 0.6}
